In [2]:
!pip install roboflow
!pip install torchvision
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 0.2.15 requires urllib3==1.26.6, but you have urllib3 1.26.13 which is incompatible.

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip


In [3]:
import sys
import os
import numpy as np
import pandas as pd

In [4]:
import sys
from PIL import Image
from roboflow import Roboflow
import torch
import wandb

print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")
print()
!ls

Python version: 3.9.8 (main, Nov 16 2021, 18:43:08) 
[GCC 7.5.0], sys.version_info(major=3, minor=9, micro=8, releaselevel='final', serial=0) 
Pytorch version: 1.10.2+cu102 

UpdatedYoloTuner.ipynb	yolov7


In [5]:
!nvidia-smi

Sat Jan  7 01:41:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:17:00.0 Off |                  N/A |
|  0%   45C    P8     8W / 260W |     10MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:18:00.0 Off |                  N/A |
|  0%   

```shell
git clone https://github.com/WongKinYiu/yolov7.git
cd yolov7/
```
### For Segmentation
```shell
git checkout u7
cd {HOME}/yolov7/seg
pip install --upgrade pip
pip install -r requirements.txt
```
### Weight Downloads

#### Pretrained

- [`yolov7.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt)
- [`yolov7x.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x.pt)
- [`yolov7-w6.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6.pt)
- [`yolov7-e6.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6.pt)
- [`yolov7-d6.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-d6.pt)
- [`yolov7-e6e.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e.pt)

#### Transfer Learning

- [`yolov7_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt)
- [`yolov7x_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7x_training.pt)
- [`yolov7-w6_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6_training.pt)
- [`yolov7-e6_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6_training.pt)
- [`yolov7-d6_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-d6_training.pt)
- [`yolov7-e6e_training.pt`](https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e_training.pt)

#### Shell Download Commands
```bash
MODEL_NAME=x
# MODEL_NAME=-e6
URL=https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7$MODEL_NAME.pt
curl -L URL -o yolov7$MODEL_NAME.pt
```



### Training
#### Distributed
```bash
python -m torch.distributed.launch \
--nproc_per_node 4 \
--master_port 9527 \

train.py \
--workers 8 \
--device 0,1,2,3 \ #must select multiple devices
--batch-size 4 \
--data data/io_segmentation4.yaml \ 
--img 1280 \
--cfg cfg/training/yolov7-e6e.yaml \ 
--weights model_weights/yolov7-e6_training.pt \ 
--name 2017_2019_2020_attempt_ \
--hyp data/hyp.scratch.custom.yaml \ 
--epochs 1200
```

#### Single GPU


#### Segmentation
```bash
python segment/train.py \
 --batch-size 2 \
 --img-size 640 \
 --epochs 1000 \
 --data data/2017_2019_2020.yaml \
 --weights yolov7$MODEL_NAME.pt \
 --device 0 \
 --name custom
 ```

In [6]:
rf = Roboflow(api_key="OYCys85yUllqTPobA5GG")
project = rf.workspace("tid-detection").project("ionospheresegmentation")
dataset = project.version(12).download("yolov7")

loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Extracting Dataset Version Zip to IonosphereSegmentation-12 in yolov7pytorch:: 1


In [7]:
%cd yolov7
!curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6e_training.pt -o yolov7-e6e_training.pt
%ls

/home/nwcallahan/yolov7updated/yolov7
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  318M  100  318M    0     0  44.3M      0  0:00:07  0:00:07 --:--:-- 45.6M
cfg/       export.py   models/         requirements.txt  train_aux.py
data/      figure/     model_weights/  runs/             train.py
datasets/  hubconf.py  paper/          scripts/          utils/
deploy/    inference/  __pycache__/    test.py           yolov7-e6e_training.pt
detect.py  LICENSE.md  README.md       tools/


In [14]:
run_str = """
python -m 
train_aux 
--workers 4 
--device 0 
--batch-size 1 
--data data/2017_2019_2020.yaml  
--img 640 
--cfg cfg/training/yolov7-e6.yaml  
--weights model_weights/yolov7-e6_training.pt  
--name 2017_2019_2020_attempt_ 
--hyp data/hyp.scratch.custom.yaml  
--epochs 1200"""
run_str = run_str.replace('\n','')
os.system(run_str)

YOLOR 🚀 v0.1-121-g89031be torch 1.12.1+cu102 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11019.5625MB)

Namespace(weights='model_weights/yolov7-e6_training.pt', cfg='cfg/training/yolov7-e6.yaml', data='data/2017_2019_2020.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=1200, batch_size=1, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=4, project='runs/train', entity=None, name='2017_2019_2020_attempt_', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/2017_2019_2020_attempt_6', total_batch_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-01-07 01:51:57.052668: I tens

 52                -1  1   2360320  models.common.Conv                      [512, 512, 3, 1]              
 53                -1  1   2360320  models.common.Conv                      [512, 512, 3, 1]              
 54                -1  1   2360320  models.common.Conv                      [512, 512, 3, 1]              
 55[-1, -3, -5, -7, -8]  1         0  models.common.Concat                    [1]                           
 56                -1  1   3279360  models.common.Conv                      [2560, 1280, 1, 1]            
 57                -1  1  11887360  models.common.SPPCSPC                   [1280, 640, 1]                
 58                -1  1    308160  models.common.Conv                      [640, 480, 1, 1]              
 59                -1  1         0  torch.nn.modules.upsampling.Upsample    [None, 2, 'nearest']          
 60                45  1    461760  models.common.Conv                      [960, 480, 1, 1]              
 61          [-1, -2]  1         0 

143                57  1   7375360  models.common.Conv                      [640, 1280, 3, 1]             
144[136, 137, 138, 139, 140, 141, 142, 143]  1    118616  models.yolo.IAuxDetect                  [1, [[19, 27, 44, 40, 38, 94], [96, 68, 86, 152, 180, 137], [140, 301, 303, 264, 238, 542], [436, 615, 739, 380, 925, 792]], [320, 640, 960, 1280, 320, 640, 960, 1280]]
Model Summary: 645 layers, 110378648 parameters, 110378648 gradients

Transferred 874/896 items from model_weights/yolov7-e6_training.pt
Scaled weight_decay = 0.0005
Optimizer groups: 153 .bias, 153 conv.weight, 153 other


wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)


train: Scanning 'datasets/2017_2019_2020/train/labels.cache' images and labels... 657 found, 0 missing, 315 empty, 0 corrupted: 100%|██████████| 657/657 [00:00<?, ?it/s]
val: Scanning 'datasets/2017_2019_2020/valid/labels.cache' images and labels... 73 found, 0 missing, 40 empty, 0 corrupted: 100%|██████████| 73/73 [00:00<?, ?it/s]
Image sizes 640 train, 640 test
Using 0 dataloader workers
Logging results to runs/train/2017_2019_2020_attempt_6
Starting training for 1200 epochs...

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
  0%|          | 0/657 [00:00<?, ?it/s]

...

 1198/1199     2.79G  0.005372  0.001715         0  0.007086         4       640: 100%|██████████| 657/657 [01:17<00:00,  8.53it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 37/37 [00:01<00:00, 35.50it/s]

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
 1199/1199     2.79G         0  0.001134         0  0.001134         0       640:   0%|          | 1/657 [00:00<01:15,  8.67it/s]

                 all          73          33       0.193      0.0909      0.0595      0.0131


 1199/1199     2.79G  0.005414   0.00178         0  0.007193         3       640: 100%|██████████| 657/657 [01:18<00:00,  8.41it/s]
               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 37/37 [00:01<00:00, 30.74it/s]
1200 epochs completed in 30.730 hours.



                 all          73          33       0.197      0.0909      0.0487      0.0112
Optimizer stripped from runs/train/2017_2019_2020_attempt_6/weights/last.pt, 221.4MB
Optimizer stripped from runs/train/2017_2019_2020_attempt_6/weights/best.pt, 221.4MB


0

In [9]:
!git checkout u7
!cd seg
!pip install --upgrade pip
!pip install -r requirements.txt
run_str = """
python -m torch.distributed.launch 
--nproc_per_node 4 
--master_port 9527 

segment/train.py 
--workers 8 
--device 0,1,2,3 
--batch-size 2 
--data data/2017_2019_2020.yaml  
--img 1280 
--cfg cfg/training/yolov7-seg.yaml  
--weights model_weights/yolov7-seg.pt  
--name 2017_2019_2020_attempt_ 
--hyp data/hyp.scratch.custom.yaml  
--epochs 1200"""

error: Your local changes to the following files would be overwritten by checkout:
	cfg/training/yolov7-e6.yaml
Please commit your changes or stash them before you switch branches.
Aborting
/bin/bash: line 0: cd: seg: No such file or directory
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:0031m15.5 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.3
    Uninstalling pip-22.3:
      Successfully uninstalled pip-22.3
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 18.4 MB/s eta 0:00:0031m18.8 MB/s eta 0:00:01


  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.10.0 requires pyyaml<6.0,>=5.1, but you have pyyaml 6.0 which is incompatible.
roboflow 0.2.15 requires urllib3==1.26.6, but you have urllib3 1.26.13 which is incompatible.


In [10]:
FINE_TUNE = True
NAME = 'yolo_tid'
DATASET = 'hard_hat'
MODEL = 'yolov7'
IMG_SIZE = (640)
DEVICE = 0
BATCH_SIZE = 8
EPOCHS = None

In [11]:
DATASET = f'data/{DATASET}.yaml'
WEIGHTS = f'{MODEL}_training.pt' if FINE_TUNE else f'{MODEL}.pt'
CFG = f'cfg/training/{MODEL}-custom.yaml' if FINE_TUNE else f'cfg/training/{MODEL}.yaml'
HYP = f'data/hyp.scratch.custom.yaml' if FINE_TUNE else f'hyp.scratch.yaml'
IMG = f'{IMG_SIZE[0]}' if len(IMG_SIZE) < 2 else f'{IMG_SIZE[0] IMG_SIZE[1]}'

command = 'python train.py --workers 2'
arg_dict = {
    '--device' : str(DEVICE),
    '--batch-size' : str(BATCH_SIZE),
    '--data' : DATASET,
    '--img' : IMG_SIZE,
    '--cfg' : CFG,
    '--weights' : WEIGHTS,
    '--name' : NAME,
    '--hyp' : HYP,   
}
if isinstance(EPOCHS, (int, float)):
    arg_dict['--epochs'] = str(EPOCHS)
if WEIGHTS not in os.listdir():
    !wget f'https://github.com/WongKinYiu/yolov7/releases/download/v0.1/{WEIGHTS}'

SyntaxError: f-string: invalid syntax (480667781.py, line 5)

In [ ]:
!# Download YOLOv7 code
!git clone https://github.com/WongKinYiu/yolov7
    
%cd yolov7
!ls

In [ ]:
!sed -i 's#./coco/val2017.txt#/content/coco/val2017.txt#g' /content/yolov7/data/coco.yaml
!cat /content/yolov7/data/coco.yaml

In [ ]:
!sed -i 's#./images/#/content/coco/images/#g' /content/coco/val2017.txt 
!cat /content/coco/val2017.txt > coco_content.txt

In [ ]:
# download weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

In [ ]:
!pip install -r requirements.txt
!pip install setuptools==59.5.0
!pip install torchvision==0.11.3+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

In [ ]:
!python detect.py --weights ./yolov7.pt --conf 0.25 --img-size 640 --source inference/images/horses.jpg

In [ ]:
print(os.listdir('..'))
!ls
os.chdir('yolov7')
!ls

In [ ]:
def conv_yolo(txt_path):
    new_txt = ''
    flag = True
    prev_digit = ''

    f = open(txt_path, 'r')
    prev_objs = list()
    for line in f.readlines():
        vals = line.replace('\n',' ').split(' ')
        assert vals[0].isdigit(), vals[0]
        if not vals[0] in prev_objs:
            if flag:
                flag = False
            else:
                new_txt += '\n'
            prev_digit = vals[0]
            prev_objs.append(vals[0])                
            new_txt += vals[0] + ' '
        new_txt += ' '.join(vals[1:])
        new_txt += ' '
    new_txt = new_txt.replace('  ',' ').replace(' \n','\n')
    new_txt = new_txt[:-1]
    print(new_txt)
    f.close()
    text_file = open(txt_path, 'w')
    text_file.write(new_txt)
    text_file.close()
    return new_txt

target_dir = 'hard_hat/train/labels/'
for txt_path in [target_dir + file for file in sorted(os.listdir(target_dir)) if file.endswith('.txt')]:
    conv_yolo(txt_path)

In [ ]:
!nvidia-smi

In [ ]:
!pip install wandb
!pip install --upgrade setuptools pip --user
!pip install --ignore-installed PyYAML
!pip install pycocotools>=2.0

In [ ]:
python -m torch.distributed.launch --nproc_per_node 4 --master_port 9527 segment/train.py --workers 8 --device 0,1,2,3^C-data 2017_2019v1/data.yaml --batch 8 --weights yolov7x-seg.pt --cfg yolov7-seg.yaml --epochs 1200 --name 2017_2019v1run1 --img 1280 --hyp data/hyps/hyp.scratch-high.yaml


In [ ]:
PROTOC_ZIP=protoc-3.9.2-linux-x86_64.zip &&
curl -OL https://github.com/protocolbuffers/protobuf/releases/download/v3.9.2/$PROTOC_ZIP &&
sudo unzip -o $PROTOC_ZIP -d /usr/local bin/protoc &&
sudo unzip -o $PROTOC_ZIP -d /usr/local 'include/*' &&
rm -f $PROTOC_ZIP
